In [1]:
import numpy as np
import pandas as pd
import re
import geopandas as gpd
from dbfread import DBF


C:\Users\chimi\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\chimi\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\chimi\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
# Clean the Cancer trial file all over us
directory = 'C:/Users/chimi/Desktop/Python Data Science Projects/detecting_cancer/data/raw_data/cancer_trial_studies.csv'
cancer_df = pd.read_csv(directory)

In [5]:
# Preview of the dataset
cancer_df.head(5)

,NCT Number,Study Title,Study URL,Acronym,Study Status,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,...,Study Design,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents
0,NCT01443026,The Effects of Lycopene on High Risk Prostatic...,https://clinicaltrials.gov/study/NCT01443026,NaN,COMPLETED,The purpose of this research study is to compa...,YES,Intraepithelial Prostatic Neoplasia|Prostatic ...,DRUG: Lycopene 30mg|DRUG: Placebo,"Tissue Biomarkers, We will use conventional im...",...,Allocation: RANDOMIZED|Intervention Model: PAR...,2005-0828|R01CA090759,2006-02,2009-04,2009-04,2011-09-29,2014-04-17,2019-11-19,"Northwestern Memorial Hospital, Chicago, Illin...",NaN
1,NCT02529852,A Phase I/II Study of Lenalidomide and Obinutu...,https://clinicaltrials.gov/study/NCT02529852,NaN,COMPLETED,There are 2 parts to this study: Part 1 (dose ...,NO,Lymphoma,DRUG: Lenalidomide|DRUG: Obinutuzumab|DRUG: Cy...,"Maximum Tolerated Dose (MTD) of Lenalidomide, ...",...,Allocation: NA|Intervention Model: SINGLE_GROU...,2015-0069|NCI-2015-01517,2015-11-04,2022-10-31,2022-10-31,2015-08-20,NaN,2022-11-03,"University of Texas MD Anderson Cancer Center,...",NaN
2,NCT01236352,Multiple Ascending Dose of BMS-911543,https://clinicaltrials.gov/study/NCT01236352,NaN,TERMINATED,The purpose of this first in human study is to...,YES,Cancer,DRUG: BMS-911543,"Number of Participants With Adverse Events, Sa...",...,Allocation: NON_RANDOMIZED|Intervention Model:...,CA215-001,2011-04-07,2015-11-19,2015-11-19,2010-11-07,2019-03-21,2019-07-31,"Mayo Clinic, Rochester, Minnesota, 55905, Unit...",NaN
3,NCT01516905,PET/CT Imaging of Malignant Brain Tumors With ...,https://clinicaltrials.gov/study/NCT01516905,NaN,COMPLETED,The purpose of this study is to evaluate diagn...,NO,Glioblastoma|Brain Metastases,DRUG: NM404,"tumor to background ratios, patients will be i...",...,Observational Model: |Time Perspective: p,RO11310|1R01CA158800-01|NCI-2012-02042|2011-04...,2011-12,2016-05-17,2016-05-17,2012-01-25,NaN,2019-11-15,"University of Wisconsin Hospital and Clinics, ...",NaN
4,NCT02293005,Phase II Trial of Alisertib (MLN8237) in Salva...,https://clinicaltrials.gov/study/NCT02293005,NaN,COMPLETED,Objectives:\n\nObjectives\n\nThe primary objec...,YES,Lung Cancer|Mesothelioma,DRUG: Alisertib,"Disease Control Rate at 4 Months, Patients who...",...,Allocation: NA|Intervention Model: SINGLE_GROU...,2014-0497|NCI-2014-02568,2015-05-20,2021-07-06,2021-07-06,2014-11-18,2022-07-06,2022-10-04,"University of Texas MD Anderson Cancer Center,...","Study Protocol and Statistical Analysis Plan, ..."


In [6]:
# Number of rows and columns
cancer_df.shape

(5480, 30)

In [7]:
# Explore structure of the data
cancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5480 entries, 0 to 5479
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   NCT Number                  5480 non-null   object 
 1   Study Title                 5480 non-null   object 
 2   Study URL                   5480 non-null   object 
 3   Acronym                     874 non-null    object 
 4   Study Status                5480 non-null   object 
 5   Brief Summary               5480 non-null   object 
 6   Study Results               5480 non-null   object 
 7   Conditions                  5480 non-null   object 
 8   Interventions               5202 non-null   object 
 9   Primary Outcome Measures    5439 non-null   object 
 10  Secondary Outcome Measures  4473 non-null   object 
 11  Other Outcome Measures      532 non-null    object 
 12  Sponsor                     5480 non-null   object 
 13  Collaborators               2489 

In [8]:
# Check for duplicate columns
cancer_df.shape[1] == len(set(cancer_df.columns))


True

In [9]:
# Check for duplicates
duplicate_rows = cancer_df[cancer_df.duplicated()]

# Count of duplicate rows
duplicate_count = duplicate_rows.shape[0]

duplicate_count

0

In [10]:
# Check for missing values in each column
missing_vals = cancer_df.isnull().sum()

# Checking the data type of each column
data_types = cancer_df.dtypes


missing_vals, data_types

(NCT Number                       0
 Study Title                      0
 Study URL                        0
 Acronym                       4606
 Study Status                     0
 Brief Summary                    0
 Study Results                    0
 Conditions                       0
 Interventions                  278
 Primary Outcome Measures        41
 Secondary Outcome Measures    1007
 Other Outcome Measures        4948
 Sponsor                          0
 Collaborators                 2991
 Sex                              0
 Age                              0
 Phases                        1565
 Enrollment                      25
 Funder Type                      0
 Study Type                       0
 Study Design                    25
 Other IDs                        0
 Start Date                      26
 Primary Completion Date         47
 Completion Date                 61
 First Posted                     0
 Results First Posted          2771
 Last Update Posted         

In [11]:
# Checking missing values in the first row of the Dataframe
missing_count = cancer_df.iloc[0].isna().sum()
print(missing_count)

3


In [12]:
# Find the zipcodes from locations
cancer_df['Locations']

0       Northwestern Memorial Hospital, Chicago, Illin...
1       University of Texas MD Anderson Cancer Center,...
2       Mayo Clinic, Rochester, Minnesota, 55905, Unit...
3       University of Wisconsin Hospital and Clinics, ...
4       University of Texas MD Anderson Cancer Center,...
                              ...                        
5475    University of California, Irvine School of Med...
5476    Tufts Medical Center, Boston, Massachusetts, 0...
5477    Montefiore Medical Center Montefiore Medical C...
5478    George Washington University, Washington, Dist...
5479    St. Jude Heritage Healthcare, Fullerton, Calif...
Name: Locations, Length: 5480, dtype: object

In [13]:
# Define the 're' expression
zip_code_pattern = r'\b\d{5}\b'

# Extracting zip codes from the 'Locations' column
cancer_df['Zip Codes'] = cancer_df['Locations'].apply(lambda x: re.findall(zip_code_pattern, str(x)))

# Displaying the first few rows to verify the extraction
extracted_zip_codes = cancer_df[['Locations', 'Zip Codes']].head()
extracted_zip_codes

,Locations,Zip Codes
0,"Northwestern Memorial Hospital, Chicago, Illin...","[60611, 60612]"
1,"University of Texas MD Anderson Cancer Center,...",[77030]
2,"Mayo Clinic, Rochester, Minnesota, 55905, Unit...","[55905, 10029, 77030]"
3,"University of Wisconsin Hospital and Clinics, ...",[53792]
4,"University of Texas MD Anderson Cancer Center,...",[77030]


In [14]:
# Length of the zipcodes
all_zip_codes = [zip_code for sublist in cancer_df['Zip Codes'] for zip_code in sublist]

# Find unique zip codes
unique_zip_codes = set(all_zip_codes)

print('Unique zip count:', len(unique_zip_codes), 'Total zipcode count:', len(all_zip_codes))


Unique zip count: 5394 Total zipcode count: 80372


In [15]:

# Include zipcodes with 0 trials
countbyZip = pd.Series(all_zip_codes).value_counts()
countbyZip = pd.DataFrame(countbyZip, columns=['zipCount'])  # Convert the Series to a DataFrame
countbyZip.reset_index(level=0, inplace=True)  # Reset the index to turn the index into a column
countbyZip.rename(columns={'index': 'zipCode'}, inplace=True)  # Rename the column 'index' to 'zipCode'

# Convert 'zipCode' column to string
countbyZip['zipCode'] = countbyZip['zipCode'].astype(str)


In [16]:
# Display the first five rows for countbyZip
countbyZip.head(5)

,zipCode,zipCount
0,77030,1081
1,10065,660
2,02215,611
3,63110,508
4,19104,396


In [17]:
# Convert it to a csv file
countbyZip.to_csv('cancertrials_zipcodes.csv',index=False)

In [18]:
# Using re to find unique zipcode for preview
# Uncomment after running
# Find the unique zipcodes

# Regular expression pattern for a 5-digit zip code
#zip_code_pattern = r'\b\d{5}\b'

# Function to extract the first found 5-digit zip code
#def extract_first_zip_code(location):
    #match = re.search(zip_code_pattern, str(location))
    #return match.group() if match else None

# Extracting the first found 5-digit zip code from each entry in the 'Locations' column of cancer_df
#cancer_df['First Zip Code'] = cancer_df['Locations'].apply(extract_first_zip_code)

# Displaying the first few rows to verify the extraction of the first zip code
#extracted_first_zip_codes = cancer_df[['Locations', 'First Zip Code']].head()
#print(extracted_first_zip_codes)

In [19]:
# Keep the unique zipcodes wbile dropping others
#cancer_df.drop(columns=['First Zip Code'], axis=1, inplace=True)

In [20]:
# Display the first 5 rows with zipcodes
cancer_df.head(5)

,NCT Number,Study Title,Study URL,Acronym,Study Status,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,...,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,Zip Codes
0,NCT01443026,The Effects of Lycopene on High Risk Prostatic...,https://clinicaltrials.gov/study/NCT01443026,NaN,COMPLETED,The purpose of this research study is to compa...,YES,Intraepithelial Prostatic Neoplasia|Prostatic ...,DRUG: Lycopene 30mg|DRUG: Placebo,"Tissue Biomarkers, We will use conventional im...",...,2005-0828|R01CA090759,2006-02,2009-04,2009-04,2011-09-29,2014-04-17,2019-11-19,"Northwestern Memorial Hospital, Chicago, Illin...",NaN,"[60611, 60612]"
1,NCT02529852,A Phase I/II Study of Lenalidomide and Obinutu...,https://clinicaltrials.gov/study/NCT02529852,NaN,COMPLETED,There are 2 parts to this study: Part 1 (dose ...,NO,Lymphoma,DRUG: Lenalidomide|DRUG: Obinutuzumab|DRUG: Cy...,"Maximum Tolerated Dose (MTD) of Lenalidomide, ...",...,2015-0069|NCI-2015-01517,2015-11-04,2022-10-31,2022-10-31,2015-08-20,NaN,2022-11-03,"University of Texas MD Anderson Cancer Center,...",NaN,[77030]
2,NCT01236352,Multiple Ascending Dose of BMS-911543,https://clinicaltrials.gov/study/NCT01236352,NaN,TERMINATED,The purpose of this first in human study is to...,YES,Cancer,DRUG: BMS-911543,"Number of Participants With Adverse Events, Sa...",...,CA215-001,2011-04-07,2015-11-19,2015-11-19,2010-11-07,2019-03-21,2019-07-31,"Mayo Clinic, Rochester, Minnesota, 55905, Unit...",NaN,"[55905, 10029, 77030]"
3,NCT01516905,PET/CT Imaging of Malignant Brain Tumors With ...,https://clinicaltrials.gov/study/NCT01516905,NaN,COMPLETED,The purpose of this study is to evaluate diagn...,NO,Glioblastoma|Brain Metastases,DRUG: NM404,"tumor to background ratios, patients will be i...",...,RO11310|1R01CA158800-01|NCI-2012-02042|2011-04...,2011-12,2016-05-17,2016-05-17,2012-01-25,NaN,2019-11-15,"University of Wisconsin Hospital and Clinics, ...",NaN,[53792]
4,NCT02293005,Phase II Trial of Alisertib (MLN8237) in Salva...,https://clinicaltrials.gov/study/NCT02293005,NaN,COMPLETED,Objectives:\n\nObjectives\n\nThe primary objec...,YES,Lung Cancer|Mesothelioma,DRUG: Alisertib,"Disease Control Rate at 4 Months, Patients who...",...,2014-0497|NCI-2014-02568,2015-05-20,2021-07-06,2021-07-06,2014-11-18,2022-07-06,2022-10-04,"University of Texas MD Anderson Cancer Center,...","Study Protocol and Statistical Analysis Plan, ...",[77030]


In [66]:
# Read the ZCTA (containing the zipcodes) as CSV
zcta_path = 'C:/Users/chimi/Downloads/tl_2020_us_zcta520/tl_2020_us_zcta520.shp'
zip_code_gdf= gpd.read_file(zcta_path)

In [67]:
# Display the first row of  zip_code_gdf
zip_code_gdf.head(5)

,ZCTA5CE20,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,35592,35592,B5,G6350,S,298552385,235989,+33.7427261,-088.0973903,"POLYGON ((-88.24735 33.65390, -88.24713 33.654..."
1,35616,35616,B5,G6350,S,559506992,41870756,+34.7395036,-088.0193814,"POLYGON ((-88.13997 34.58184, -88.13995 34.582..."
2,35621,35621,B5,G6350,S,117838488,409438,+34.3350314,-086.7270557,"POLYGON ((-86.81659 34.34960, -86.81649 34.349..."
3,35651,35651,B5,G6350,S,104521045,574316,+34.4609087,-087.4801507,"POLYGON ((-87.53087 34.42492, -87.53082 34.429..."
4,36010,36010,B5,G6350,S,335675180,236811,+31.6598950,-085.8128958,"POLYGON ((-85.95712 31.67744, -85.95676 31.677..."


In [68]:
# Count the number of null values in the 'ZCTA5CE20' column of zip_code_gdf
sum(zip_code_gdf['ZCTA5CE20'].isnull())

0

In [69]:
# Rename the columns
columns = {'ZCTA5CE20':'zipCode'}
zip_code_gdf.rename(columns=columns, inplace=True)

In [70]:
# Convert 'zipCode'  to string
zip_code_gdf['zipCode'] = zip_code_gdf['zipCode'].astype('string')


In [71]:
# Read the countyfile 
countyfile_path = 'C:/Users/chimi/Desktop/Python Data Science Projects/detecting_cancer/Data/cb_2022_us_county_500k/cb_2022_us_county_500k.shp'
county_gpd = gpd.read_file(countyfile_path)

In [72]:
# Display the first five rows of county_gpd
county_gpd.head(5)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,01,069,00161560,0500000US01069,01069,Houston,Houston County,AL,Alabama,06,1501742235,4795415,"POLYGON ((-85.71209 31.19727, -85.70934 31.198..."
1,01,023,00161537,0500000US01023,01023,Choctaw,Choctaw County,AL,Alabama,06,2365900083,19114321,"POLYGON ((-88.47323 31.89386, -88.46888 31.930..."
2,01,005,00161528,0500000US01005,01005,Barbour,Barbour County,AL,Alabama,06,2292160151,50523213,"POLYGON ((-85.74803 31.61918, -85.74544 31.618..."
3,01,107,00161580,0500000US01107,01107,Pickens,Pickens County,AL,Alabama,06,2282835044,22621093,"POLYGON ((-88.34043 32.99120, -88.33101 33.072..."
4,01,033,00161542,0500000US01033,01033,Colbert,Colbert County,AL,Alabama,06,1535742270,79160396,"POLYGON ((-88.13925 34.58779, -88.13872 34.589..."


In [73]:
# Number of rows and columns
county_gpd.shape

(3235, 13)

In [74]:
# Check for missing values
county_gpd.isnull().sum()

STATEFP       0
COUNTYFP      0
COUNTYNS      0
AFFGEOID      0
GEOID         0
NAME          0
NAMELSAD      0
STUSPS        0
STATE_NAME    0
LSAD          0
ALAND         0
AWATER        0
geometry      0
dtype: int64

In [75]:
# Drop redunant column
county_gpd.drop(columns=['NAME'], inplace=True)


In [76]:
# Load your ZIP code data into a GeoDataFrame

# Perform a spatial join
combined_gdf = gpd.sjoin(zip_code_gdf, county_gpd, how='left', op='intersects')


C:\Users\chimi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [77]:
#Number of rows and columns after spatial join
combined_gdf.shape

(69318, 22)

In [78]:
# Display the new dataframe
print(combined_gdf)

      zipCode GEOID20 CLASSFP20 MTFCC20 FUNCSTAT20    ALAND20  AWATER20  \
0       35592   35592        B5   G6350          S  298552385    235989   
1       35616   35616        B5   G6350          S  559506992  41870756   
1       35616   35616        B5   G6350          S  559506992  41870756   
1       35616   35616        B5   G6350          S  559506992  41870756   
1       35616   35616        B5   G6350          S  559506992  41870756   
...       ...     ...       ...     ...        ...        ...       ...   
33786   10540   10540        B5   G6350          S    1394970     27777   
33787   23081   23081        B5   G6350          S    7753698  14654483   
33788   23147   23147        B5   G6350          S     143646         0   
33789   23298   23298        B5   G6350          S     288664         0   
33790   12544   12544        B5   G6350          S    1486252         0   

        INTPTLAT20    INTPTLON20  \
0      +33.7427261  -088.0973903   
1      +34.7395036  -088.01

In [79]:
# Rename a column
combined_gdf.rename(columns={'NAMELSAD':'countyName'}, inplace=True)

In [80]:
# Check for missing 'zipcodes'
sum(combined_gdf['zipCode'].isnull())

0

In [81]:
# Rename the columns
columns ={'GEOID':'countyCode'}
combined_gdf.rename(columns=columns, inplace=True)

In [82]:
# Find zip codes present and not present in fullZips
zip_codes_present = countbyZip[countbyZip['zipCode'].isin(combined_gdf['zipCode'])]
zip_codes_absent = countbyZip[~countbyZip['zipCode'].isin(combined_gdf['zipCode'])]

In [83]:
# Convert 'zipcode' and 'countycode' to string
combined_gdf['zipCode'] = combined_gdf['zipCode'].astype('string')
combined_gdf['countyCode'] = combined_gdf['countyCode'].astype('string')

# Print the number of zip codes present and absent in combined_gdf
print("# of zip codes in countByZip that are in combined_gdf:", len(zip_codes_present))
print("# not in combined_gdf:", len(zip_codes_absent))


# Merge countByZip into fullZips
merged_gdf = pd.merge(combined_gdf, countbyZip, how='left', on='zipCode')


# Fill missing values and rename column
merged_gdf['zipCount'].fillna(0, inplace=True)
merged_gdf.rename(columns={'zipCount':'studyCount'}, inplace=True)

# of zip codes in countByZip that are in combined_gdf: 3775
# not in combined_gdf: 1619


In [84]:
#Print summary statistics
print('combined merged:')
print("# unique zips:", merged_gdf['zipCode'].nunique())
print("# of duplicate zips:", merged_gdf['zipCode'].duplicated().sum())
print("# zips with > 0 clin trials:", (merged_gdf['studyCount'] > 0).sum())
print("# zips with 0 clin trials:", (merged_gdf['studyCount'] == 0).sum())
print("Columns in merged fullZips:", merged_gdf.columns.tolist())

combined merged:
# unique zips: 33791
# of duplicate zips: 35527
# zips with > 0 clin trials: 6523
# zips with 0 clin trials: 62795
Columns in merged fullZips: ['zipCode', 'GEOID20', 'CLASSFP20', 'MTFCC20', 'FUNCSTAT20', 'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20', 'geometry', 'index_right', 'STATEFP', 'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'countyCode', 'countyName', 'STUSPS', 'STATE_NAME', 'LSAD', 'ALAND', 'AWATER', 'studyCount']


In [85]:
# Display the first 5 rows of merged_gdf
merged_gdf.head(5)

,zipCode,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry,...,COUNTYNS,AFFGEOID,countyCode,countyName,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,studyCount
0,35592,35592,B5,G6350,S,298552385,235989,+33.7427261,-088.0973903,"POLYGON ((-88.24735 33.65390, -88.24713 33.654...",...,00161563,0500000US01075,01075,Lamar County,AL,Alabama,06,1566544672,1600679,0.0
1,35616,35616,B5,G6350,S,559506992,41870756,+34.7395036,-088.0193814,"POLYGON ((-88.13997 34.58184, -88.13995 34.582...",...,00161555,0500000US01059,01059,Franklin County,AL,Alabama,06,1641945980,32539346,0.0
2,35616,35616,B5,G6350,S,559506992,41870756,+34.7395036,-088.0193814,"POLYGON ((-88.13997 34.58184, -88.13995 34.582...",...,00161542,0500000US01033,01033,Colbert County,AL,Alabama,06,1535742270,79160396,0.0
3,35616,35616,B5,G6350,S,559506992,41870756,+34.7395036,-088.0193814,"POLYGON ((-88.13997 34.58184, -88.13995 34.582...",...,00161564,0500000US01077,01077,Lauderdale County,AL,Alabama,06,1730150450,133604582,0.0
4,35616,35616,B5,G6350,S,559506992,41870756,+34.7395036,-088.0193814,"POLYGON ((-88.13997 34.58184, -88.13995 34.582...",...,00695791,0500000US28141,28141,Tishomingo County,MS,Mississippi,06,1098925142,52373918,0.0


In [86]:
# Read the income file downloaded from census.gov
file_path = 'C:/Users/chimi/Downloads/income.xls'


income_df = pd.read_excel(file_path, header=[3])



In [87]:
# Display the first five rows
income_df.head(5)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound.1,90% CI Upper Bound.1,...,90% CI Upper Bound.5,Median Household Income,90% CI Lower Bound.6,90% CI Upper Bound.6,"Poverty Estimate, Age 0-4",90% CI Lower Bound.7,90% CI Upper Bound.7,"Poverty Percent, Age 0-4",90% CI Lower Bound.8,90% CI Upper Bound.8
0,0,0,US,United States,40951625,40691315,41211935,12.6,12.5,12.7,...,15.7,74755,74607,74903,3141107,3093587,3188627,17.4,17.1,17.7
1,1,0,AL,Alabama,798469,780933,816005,16.2,15.8,16.6,...,21.6,59703,58986,60420,68982,64247,73717,24.4,22.7,26.1
2,1,1,AL,Autauga County,6988,5522,8454,11.8,9.3,14.3,...,19.5,70148,62906,77390,.,.,.,.,.,.
3,1,3,AL,Baldwin County,30195,26604,33786,12.4,10.9,13.9,...,17.2,71704,66180,77228,.,.,.,.,.,.
4,1,5,AL,Barbour County,5860,4715,7005,26.7,21.5,31.9,...,44.4,41151,36595,45707,.,.,.,.,.,.


In [88]:
# Reset the index first to ensure it's unique
income_df = income_df.reset_index(drop=True)

# Now drop the first row
income_df = income_df.drop(income_df.index[0])

In [89]:
# Rename the columns
columns = {'State FIPS Code':'SFIPS','County FIPS Code':'CFIPS',
           'Name':'countyName','Postal Code':'STATE_NAME','Poverty Percent, All Ages':'PovertyPercent',
           'Poverty Estimate, All Ages':'PovertyEst',
           'Median Household Income':'medIncome'}

income_df.rename(columns=columns, inplace=True)

In [90]:
# Convert SFIPS and CFIPS to strings
income_df['SFIPS'] = income_df['SFIPS'].astype('string')
income_df['CFIPS'] = income_df['CFIPS'].astype('string')

In [91]:
# Format the SIPS and FIPS 

for i, j in enumerate(income_df['SFIPS']):
    if len(j) == 1:
        income_df.loc[i, 'SFIPS'] = '0{}'.format(j)

        
for i, j in enumerate(income_df['CFIPS']):
    if len(j) == 1:
        income_df.loc[i, 'CFIPS'] = '00{}'.format(j)
    elif len(j) == 2:
        income_df.loc[i, 'CFIPS'] = '00{}'.format(j)


In [92]:
# Create a new column called countryCode and format it
income_df['countyCode'] = np.nan

# Concatenate 'SFIPS and CFIPS values into the countryCode'

for i, j in enumerate(income_df['CFIPS']):
    income_df.loc[i, 'countyCode'] = '{}{}'.format(income_df.loc[i, 'SFIPS'], j)

In [93]:
# Reset the index
income_df = income_df.reset_index(drop=True)

In [94]:
# Save the only columns we need 
income_df = income_df[['PovertyEst', 'PovertyPercent', 'medIncome', 'countyCode', 'countyName','STATE_NAME']]
# Printing information about the DataFrame
print("income_df: \n", income_df.shape)  # Display the shape of the DataFrame

# Calculating the number of unique and duplicate county codes
unique_counties = len(income_df['countyCode'].unique())
duplicates = income_df['countyCode'].duplicated().sum()  # Count of duplicated county codes

# Printing the number of unique and duplicate county codes
print("# unique counties: ", unique_counties)
print("# of duplicate counties: {}".format(duplicates))

# Printing the column names of the DataFrame
print("Columns in income_df: ", income_df.columns)

income_df: 
 (3196, 6)
# unique counties:  3195
# of duplicate counties: 1
Columns in income_df:  Index(['PovertyEst', 'PovertyPercent', 'medIncome', 'countyCode', 'countyName',
       'STATE_NAME'],
      dtype='object')


In [96]:
# Population by County
file_path = 'C:/Users/chimi/Desktop/Python Data Science Projects/detecting_cancer/data/raw_data/total_populatbycounty.csv'

pop_df = pd.read_csv(file_path)

In [97]:
# Display the first five rows of pop_df
pop_df.head(5)

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,...,RDEATH2021,RDEATH2022,RNATURALCHG2021,RNATURALCHG2022,RINTERNATIONALMIG2021,RINTERNATIONALMIG2022,RDOMESTICMIG2021,RDOMESTICMIG2022,RNETMIG2021,RNETMIG2022
0,40,3,6,1,0,Alabama,Alabama,5024356,5031362,5049846,...,13.699945,13.210008,-2.369557,-1.696934,0.358092,0.908126,5.496167,5.651639,5.854259,6.559766
1,50,3,6,1,1,Alabama,Autauga County,58802,58902,59210,...,11.548361,11.313872,0.203197,0.605200,0.253996,0.453900,4.063939,8.540040,4.317936,8.993940
2,50,3,6,1,3,Alabama,Baldwin County,231761,233219,239361,...,12.882475,11.976220,-2.865123,-2.050243,0.444369,1.296841,29.493419,28.744576,29.937788,30.041417
3,50,3,6,1,5,Alabama,Barbour County,25224,24960,24539,...,15.475060,15.108133,-4.323320,-3.858260,0.000000,0.162453,-12.444696,9.869022,-12.444696,10.031475
4,50,3,6,1,7,Alabama,Bibb County,22300,22183,22370,...,14.275133,14.467606,-3.321886,-3.290141,0.044890,0.090141,11.626602,-13.566197,11.671492,-13.476056


In [98]:
# Display the number of rows and columns
pop_df.shape

(3195, 51)

In [99]:
# Display the structure of the dataframe
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3195 entries, 0 to 3194
Data columns (total 51 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   SUMLEV                 3195 non-null   int64  
 1   REGION                 3195 non-null   int64  
 2   DIVISION               3195 non-null   int64  
 3   STATE                  3195 non-null   int64  
 4   COUNTY                 3195 non-null   int64  
 5   STNAME                 3195 non-null   object 
 6   CTYNAME                3195 non-null   object 
 7   ESTIMATESBASE2020      3195 non-null   int64  
 8   POPESTIMATE2020        3195 non-null   int64  
 9   POPESTIMATE2021        3195 non-null   int64  
 10  POPESTIMATE2022        3195 non-null   int64  
 11  NPOPCHG2020            3195 non-null   int64  
 12  NPOPCHG2021            3195 non-null   int64  
 13  NPOPCHG2022            3195 non-null   int64  
 14  BIRTHS2020             3195 non-null   int64  
 15  BIRT

In [100]:
# Rename STATE and County Columns
pop_df.rename(columns={'STATE':'SFIPS','COUNTY':'CFIPS','CTYNAME':'countyName','STNAME':'STATE_NAME'},inplace=True)

In [101]:
# Convert SFIPS and CFIPS columns to string
pop_df['SFIPS'] = pop_df['SFIPS'].astype('string')
pop_df['CFIPS'] = pop_df['CFIPS'].astype('string')

In [102]:
# Reformat SFIPS columns
for i, j in enumerate(pop_df['SFIPS']):
    if len(j) == 1:
        pop_df.loc[i, 'SFIPS'] = f'0{j}'

In [103]:
# Reformat the CFIPS column
for i, j in enumerate(pop_df['CFIPS']):
    pop_df.loc[i, 'countyCode'] = f'{pop_df.loc[i, "SFIPS"]}{pop_df.loc[i, "CFIPS"]}'


In [104]:
# Drop some of the columns 
pop_df.drop(['SFIPS','CFIPS'],inplace=True, axis=1)
print(pop_df)
print(pop_df.shape)

      SUMLEV  REGION  DIVISION STATE_NAME         countyName  \
0         40       3         6    Alabama            Alabama   
1         50       3         6    Alabama     Autauga County   
2         50       3         6    Alabama     Baldwin County   
3         50       3         6    Alabama     Barbour County   
4         50       3         6    Alabama        Bibb County   
...      ...     ...       ...        ...                ...   
3190      50       4         8    Wyoming  Sweetwater County   
3191      50       4         8    Wyoming       Teton County   
3192      50       4         8    Wyoming       Uinta County   
3193      50       4         8    Wyoming    Washakie County   
3194      50       4         8    Wyoming      Weston County   

      ESTIMATESBASE2020  POPESTIMATE2020  POPESTIMATE2021  POPESTIMATE2022  \
0               5024356          5031362          5049846          5074296   
1                 58802            58902            59210            59759 

In [105]:
# Print the duplicates
d = len([i for i in pop_df.countyCode.duplicated() if i])
print(f"# of duplicate counties: {d}")
print(f"# of unique counties: {len(pop_df['countyCode'].unique())}")
print(pop_df.columns)

# of duplicate counties: 0
# of unique counties: 3195
Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE_NAME', 'countyName',
       'ESTIMATESBASE2020', 'POPESTIMATE2020', 'POPESTIMATE2021',
       'POPESTIMATE2022', 'NPOPCHG2020', 'NPOPCHG2021', 'NPOPCHG2022',
       'BIRTHS2020', 'BIRTHS2021', 'BIRTHS2022', 'DEATHS2020', 'DEATHS2021',
       'DEATHS2022', 'NATURALCHG2020', 'NATURALCHG2021', 'NATURALCHG2022',
       'INTERNATIONALMIG2020', 'INTERNATIONALMIG2021', 'INTERNATIONALMIG2022',
       'DOMESTICMIG2020', 'DOMESTICMIG2021', 'DOMESTICMIG2022', 'NETMIG2020',
       'NETMIG2021', 'NETMIG2022', 'RESIDUAL2020', 'RESIDUAL2021',
       'RESIDUAL2022', 'GQESTIMATESBASE2020', 'GQESTIMATES2020',
       'GQESTIMATES2021', 'GQESTIMATES2022', 'RBIRTH2021', 'RBIRTH2022',
       'RDEATH2021', 'RDEATH2022', 'RNATURALCHG2021', 'RNATURALCHG2022',
       'RINTERNATIONALMIG2021', 'RINTERNATIONALMIG2022', 'RDOMESTICMIG2021',
       'RDOMESTICMIG2022', 'RNETMIG2021', 'RNETMIG2022', 'countyCode'],
      

In [106]:
# Merging two DataFrames 'income_df' and 'pop_df' based on the 'countyCode' column using an inner join.
cdf = pd.merge(income_df, pop_df[['countyCode', 'POPESTIMATE2022']], how='inner', on='countyCode')


# Print the rows and columns shape of the new dataframe
print(cdf.shape) #1243,7

# Printing the resulting merged_df1
print(cdf.columns)


(1243, 7)
Index(['PovertyEst', 'PovertyPercent', 'medIncome', 'countyCode', 'countyName',
       'STATE_NAME', 'POPESTIMATE2022'],
      dtype='object')


In [107]:
# Preview the cdf columns
cdf.head(5)

,PovertyEst,PovertyPercent,medIncome,countyCode,countyName,STATE_NAME,POPESTIMATE2022
0,77450,19.2,54315,0199,Mobile County,AL,19404
1,3963,20.7,47498,01101,Monroe County,AL,226361
2,39991,18.4,56862,01103,Montgomery County,AL,124211
3,15935,13.1,62260,01105,Morgan County,AL,8035
4,2714,35.9,33585,01107,Perry County,AL,18697


In [108]:
# Dropping STATE_NAME AND countyName before joining
# Create duplicate columns/avoid redunancy
cdf.drop(columns=['countyName','STATE_NAME'],inplace=True)

In [109]:
# Merge merged_gdf and cdf as one dataframe
data = pd.merge(merged_gdf, cdf, how='inner', on=['countyCode'])

In [110]:
# Display the first 5 rows of new columns
data.head(5)

,zipCode,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry,...,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,studyCount,PovertyEst,PovertyPercent,medIncome,POPESTIMATE2022
0,35616,35616,B5,G6350,S,559506992,41870756,+34.7395036,-088.0193814,"POLYGON ((-88.13997 34.58184, -88.13995 34.582...",...,MS,Mississippi,06,1098925142,52373918,0.0,3335,16,52635,18619
1,38827,38827,B5,G6350,S,89329999,32145,+34.5112270,-088.2423690,"POLYGON ((-88.29393 34.51418, -88.29372 34.514...",...,MS,Mississippi,06,1098925142,52373918,0.0,3335,16,52635,18619
2,38326,38326,B5,G6350,S,110288888,16400566,+35.0457825,-088.2782793,"POLYGON ((-88.36374 35.03441, -88.36369 35.034...",...,MS,Mississippi,06,1098925142,52373918,0.0,3335,16,52635,18619
3,38846,38846,B5,G6350,S,89726804,143777,+34.8517197,-088.3853894,"POLYGON ((-88.46378 34.81030, -88.46350 34.810...",...,MS,Mississippi,06,1098925142,52373918,0.0,3335,16,52635,18619
4,38873,38873,B5,G6350,S,240605312,13375617,+34.6648698,-088.2513475,"POLYGON ((-88.37022 34.70674, -88.36989 34.706...",...,MS,Mississippi,06,1098925142,52373918,0.0,3335,16,52635,18619


In [111]:
# Displaying the columns after the names
data.columns

Index(['zipCode', 'GEOID20', 'CLASSFP20', 'MTFCC20', 'FUNCSTAT20', 'ALAND20',
       'AWATER20', 'INTPTLAT20', 'INTPTLON20', 'geometry', 'index_right',
       'STATEFP', 'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'countyCode',
       'countyName', 'STUSPS', 'STATE_NAME', 'LSAD', 'ALAND', 'AWATER',
       'studyCount', 'PovertyEst', 'PovertyPercent', 'medIncome',
       'POPESTIMATE2022'],
      dtype='object')

In [112]:
# Convert 'medIncome' and 'PovertyEst' to string and remove commas
data['medIncome'] = data['medIncome'].astype(str).str.replace(',', '')
data['PovertyEst'] = data['PovertyEst'].astype(str).str.replace(',', '')

# Convert 'PovertyEst' to float, replace errors with 0
data['PovertyEst'] = pd.to_numeric(data['PovertyEst'], errors='coerce').fillna(0)

# Replace 0 values in 'PovertyEst' with the column's mean
mean_poverty_est = data['PovertyEst'].replace(0, np.nan).mean()
data['PovertyEst'] = data['PovertyEst'].replace(0, mean_poverty_est)

# Convert 'PovertyPercent' to float, replace errors with 0
data['PovertyPercent'] = pd.to_numeric(data['PovertyPercent'], errors='coerce').fillna(0)


# Print summary information about the data DataFrame
print("data:")
print(data.shape)
print("# of unique zips:", len(data['zipCode'].unique()))
print("# of unique counties:", len(data['countyCode'].unique()))
print("# of duplicate zips:", data['zipCode'].duplicated().sum())
print("# of duplicate counties:", data['countyCode'].duplicated().sum())
print("Columns in data:", data.columns)

# Select specific columns for the final DataFrame
data = data[['zipCode', 'countyCode', 'studyCount', 'STATE_NAME', 'PovertyEst', 'PovertyPercent', 'medIncome','countyName', 'POPESTIMATE2022']]


data:
(23399, 27)
# of unique zips: 15724
# of unique counties: 1208
# of duplicate zips: 7675
# of duplicate counties: 22191
Columns in data: Index(['zipCode', 'GEOID20', 'CLASSFP20', 'MTFCC20', 'FUNCSTAT20', 'ALAND20',
       'AWATER20', 'INTPTLAT20', 'INTPTLON20', 'geometry', 'index_right',
       'STATEFP', 'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'countyCode',
       'countyName', 'STUSPS', 'STATE_NAME', 'LSAD', 'ALAND', 'AWATER',
       'studyCount', 'PovertyEst', 'PovertyPercent', 'medIncome',
       'POPESTIMATE2022'],
      dtype='object')


In [113]:
# New Dataframe data columns
data.columns

Index(['zipCode', 'countyCode', 'studyCount', 'STATE_NAME', 'PovertyEst',
       'PovertyPercent', 'medIncome', 'countyName', 'POPESTIMATE2022'],
      dtype='object')

In [114]:
# Display the first five rows of the dataframe data
data.head(5)

,zipCode,countyCode,studyCount,STATE_NAME,PovertyEst,PovertyPercent,medIncome,countyName,POPESTIMATE2022
0,35616,28141,0.0,Mississippi,3335,16.0,52635,Tishomingo County,18619
1,38827,28141,0.0,Mississippi,3335,16.0,52635,Tishomingo County,18619
2,38326,28141,0.0,Mississippi,3335,16.0,52635,Tishomingo County,18619
3,38846,28141,0.0,Mississippi,3335,16.0,52635,Tishomingo County,18619
4,38873,28141,0.0,Mississippi,3335,16.0,52635,Tishomingo County,18619


In [115]:
# Cancer death rates by country
file_path = 'C:/Users/chimi/Desktop/Python Data Science Projects\detecting_cancer/data/raw_data/incd.csv'

incd = pd.read_csv(file_path, skiprows=8)

incd.head(5)

,County,FIPS,"Age-Adjusted Incidence Rate([rate note]) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,CI*Rank([rank note]),Lower CI (CI*Rank),Upper CI (CI*Rank),Average Annual Count,Recent Trend,Recent 5-Year Trend ([trend note]) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1
0,US (SEER+NPCR)(1),0,442.3,442,442.6,N/A,N/A,N/A,1698328,stable,-0.3,-0.6,0.1
1,"Union County, Florida(6)",12125,1237.4,1165.6,1312.8,N/A,1,1,237,stable,0.6,-0.5,1.9
2,"Palo Alto County, Iowa(7)",19147,658.1,591.1,731.1,N/A,1,6,82,rising,4.8,0.2,15.4
3,"Treasure County, Montana(6)",30103,652.2,401,1007.4,N/A,1,55,7,stable,-1.1,-5.6,3.3
4,"Polk County, Texas(7)",48373,633.6,604.6,663.7,N/A,1,4,425,rising,2.2,1.2,4.2


In [116]:
# Drop data for the entire USA
incd.drop(index=0, inplace=True)

In [117]:
# Number of rows and columns
incd.shape

(3143, 13)

In [118]:
# Dataframe columns
print(incd.columns)

Index(['County', ' FIPS',
       'Age-Adjusted Incidence Rate([rate note]) - cases per 100,000',
       'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval',
       'CI*Rank([rank note])', 'Lower CI (CI*Rank)', 'Upper CI (CI*Rank)',
       'Average Annual Count', 'Recent Trend',
       'Recent 5-Year Trend ([trend note]) in Incidence Rates',
       'Lower 95% Confidence Interval.1', 'Upper 95% Confidence Interval.1'],
      dtype='object')


In [119]:
# Remove extra whitespaces from columns
incd.columns = incd.columns.str.strip()

In [120]:
# Rename FIPS to CountyCode
incd.rename(columns={'FIPS':'countyCode','Recent Trend':'recTrend'}, inplace=True)

In [121]:
# Drop county names as it will create duplicate columns
# Merging this file based on matching county codes
incd.drop(columns=['County'], inplace=True)

In [122]:
# Print the unique number of counties
print('# of unique counties: ', len(incd['countyCode'].unique()))

# of unique counties:  3143


In [123]:
# Check to see if any county code is missing
sum(incd['countyCode'].isnull())

0

In [124]:
# Drop any missing 'countyCode' that are misssing
incd.dropna(subset=['countyCode'], inplace=True)

In [125]:
# Convert 'countyCode' to integers
incd['countyCode'] = incd['countyCode'].apply(lambda x: int(x))
# Convert 'countyCode' 'back to string
incd['countyCode'] = incd['countyCode'].astype('string')
 
# Rename the '5YearTrend' column to 'fiveYearTrend'
incd.rename(columns={'5YearTrend':'fiveYearTrend'}, inplace=True)


In [126]:
# Calculate the number of unique counties in the 'countyCode' column
unique_counties = incd['countyCode'].nunique()

In [127]:
# Print the unique counties and shape of the Dataframe
print('data: n# of unique counties', unique_counties)
print('DataFrame shape', incd.shape)

data: n# of unique counties 3143
DataFrame shape (3143, 12)


In [128]:
# Merge data with incd(cancer_incidentbycounty)
data = pd.merge(data, incd, how='inner', on=['countyCode'])
print(data.shape)
print(data.columns)
print(data.head(5))

(21475, 20)
Index(['zipCode', 'countyCode', 'studyCount', 'STATE_NAME', 'PovertyEst',
       'PovertyPercent', 'medIncome', 'countyName', 'POPESTIMATE2022',
       'Age-Adjusted Incidence Rate([rate note]) - cases per 100,000',
       'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval',
       'CI*Rank([rank note])', 'Lower CI (CI*Rank)', 'Upper CI (CI*Rank)',
       'Average Annual Count', 'recTrend',
       'Recent 5-Year Trend ([trend note]) in Incidence Rates',
       'Lower 95% Confidence Interval.1', 'Upper 95% Confidence Interval.1'],
      dtype='object')
  zipCode countyCode  studyCount   STATE_NAME  PovertyEst  PovertyPercent  \
0   35616      28141         0.0  Mississippi        3335            16.0   
1   38827      28141         0.0  Mississippi        3335            16.0   
2   38326      28141         0.0  Mississippi        3335            16.0   
3   38846      28141         0.0  Mississippi        3335            16.0   
4   38873      28141         0.0

In [132]:
# Cancer death rate by county
file_path = 'C:/Users/chimi/Desktop/Python Data Science Projects/detecting_cancer/data/raw_data/cancer_death.csv'
mort_df = pd.read_csv(file_path, skiprows=8)

In [133]:
# Display the first 5 rows of mort_df
mort_df.head(5)

,County,FIPS,Met Healthy People Objective of 122.7?,"Age-Adjusted Death Rate([rate note]) - deaths per 100,000",Lower CI (Rate),Upper CI (Rate),CI*Rank([rank note]),Lower CI (CI*Rank),Upper CI (CI*Rank),Average Annual Count,Recent Trend,Recent 5-Year Trend ([trend note]) in Death Rates,Lower 95% Confidence Interval,Upper 95% Confidence Interval
0,United States,0,No,149.4,149.3,149.6,N/A,N/A,N/A,599666,falling,-2,-2.2,-1.8
1,"Union County, Florida",12125,No,414.5,372.8,459.9,N/A,1,1,77,stable,-0.1,-0.9,0.7
2,"Galax City, Virginia",51640,No,363.1,307.7,426.4,N/A,1,3,33,falling,-17,-30.7,-0.6
3,"Kingman County, Kansas",20095,No,353.6,301.6,412.9,N/A,1,3,38,rising,17.7,4.3,32.9
4,"Hanson County, South Dakota",46061,No,302.1,213.4,413.3,N/A,1,24,9,rising,1.9,0.3,3.5


In [134]:
# Number of rows and columns
mort_df.shape

(3142, 14)

In [135]:
# Columns of mort_df
mort_df.columns

Index(['County', ' FIPS', 'Met Healthy People Objective of 122.7?',
       'Age-Adjusted Death Rate([rate note]) - deaths per 100,000',
       'Lower CI (Rate)', 'Upper CI (Rate)', 'CI*Rank([rank note])',
       'Lower CI (CI*Rank)', 'Upper CI (CI*Rank)', 'Average Annual Count',
       'Recent Trend', 'Recent 5-Year Trend ([trend note]) in Death Rates',
       'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval'],
      dtype='object')

In [136]:
# Remove extra whitespaces from columns
mort_df.columns = mort_df.columns.str.strip()

In [137]:
# Dropping another county names to avoid duplicate columns- as we already have the corresponding names
mort_df.drop(columns=['County'], inplace=True)

In [138]:
# Rename the columns
new_column_names = {
    'FIPS': 'countyCode',
    'Met Healthy People Objective of 122.7?': 'metObj',
    'Age-Adjusted Death Rate([rate note]) - deaths per 100,000': 'deathRate',
    'Lower CI (Rate)': 'l95DR',
    'Upper CI (Rate)': 'u95DR',
    'Average Annual Count': 'avgDeathsPerYear',
    'Recent Trend': 'recTrend',
    'Recent 5-Year Trend ([trend note]) in Death Rates': 'fiveYearTrend',
    'Lower 95% Confidence Interval': 'l95T',
    'Upper 95% Confidence Interval': 'u95T'
}

mort_df= mort_df.rename(columns=new_column_names)

# Displaying the first few rows of the dataset with renamed columns
mort_df.head()

,countyCode,metObj,deathRate,l95DR,u95DR,CI*Rank([rank note]),Lower CI (CI*Rank),Upper CI (CI*Rank),avgDeathsPerYear,recTrend,fiveYearTrend,l95T,u95T
0,0,No,149.4,149.3,149.6,N/A,N/A,N/A,599666,falling,-2,-2.2,-1.8
1,12125,No,414.5,372.8,459.9,N/A,1,1,77,stable,-0.1,-0.9,0.7
2,51640,No,363.1,307.7,426.4,N/A,1,3,33,falling,-17,-30.7,-0.6
3,20095,No,353.6,301.6,412.9,N/A,1,3,38,rising,17.7,4.3,32.9
4,46061,No,302.1,213.4,413.3,N/A,1,24,9,rising,1.9,0.3,3.5


In [139]:
# Drop rows with missing values in mort_df
mort_df.dropna(inplace=True)

# Convert 'countyCode' to integer, filter rows, and reset index
mort_df = mort_df[mort_df['countyCode'].astype('int64') > 0].reset_index(drop=True)

# Convert 'countyCode' to string and apply formatting
mort_df['countyCode'] = mort_df['countyCode'].astype('string')

# Select and retain specific columns in mort_df
selected_columns = ['countyCode', 'deathRate', 'avgDeathsPerYear', 'recTrend']
mort_df = mort_df[selected_columns]


In [140]:
# Replace '*' '' to indicate empty rows
mort_df['avgDeathsPerYear'] = mort_df['avgDeathsPerYear'].str.replace('*', '')
mort_df['recTrend'] = mort_df['recTrend'].str.replace('*', '')

C:\Users\chimi\AppData\Local\Temp\ipykernel_18828\389774492.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  mort_df['avgDeathsPerYear'] = mort_df['avgDeathsPerYear'].str.replace('*', '')
C:\Users\chimi\AppData\Local\Temp\ipykernel_18828\389774492.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  mort_df['recTrend'] = mort_df['recTrend'].str.replace('*', '')


In [141]:
# Initialize the dictionary with default empty lists for each column
error_indicies = {'deathRate': [], 'avgDeathsPerYear': []}

for col in ['deathRate', 'avgDeathsPerYear']:
    # Convert column to numeric, coercing errors to NaN
    mort_df[col] = pd.to_numeric(mort_df[col], errors='coerce')
    
    # Find rows where conversion resulted in NaN and update the dictionary
    error_indicies[col].extend(mort_df[mort_df[col].isna()].index.tolist())

# Aggregate errors from both columns
aggregated_errors = set(error_indicies['deathRate']) | set(error_indicies['avgDeathsPerYear'])

# Print the number of counties with no reported deathRate data
print("# counties with no reported deathRate data:", len(aggregated_errors))


# Drop rows with conversion errors
mort_df.drop(list(aggregated_errors), inplace=True)


# counties with no reported deathRate data: 54


In [142]:
# Convert 'deathRate' and 'avgDeathsPerYear' to float'
mort_df['deathRate'] = pd.to_numeric(mort_df['deathRate'], errors='coerce').astype('float64')
mort_df['avgDeathsPerYear'] = pd.to_numeric(mort_df['avgDeathsPerYear'], errors='coerce').astype('float64')

In [143]:
# Final preview of mort_df
mort_df.head(20)

,countyCode,deathRate,avgDeathsPerYear,recTrend
0,12125,414.5,77.0,stable
1,51640,363.1,33.0,falling
2,20095,353.6,38.0,rising
3,46061,302.1,9.0,rising
4,31115,294.0,3.0,
5,16025,283.5,4.0,
6,22035,277.7,22.0,stable
7,51690,273.9,48.0,rising
8,21193,267.4,92.0,stable
9,51580,267.3,22.0,stable


In [144]:
# Merge on matching countyCode
data = pd.merge(data, mort_df, how='inner', on=['countyCode'])
# new dataframe
print(data)
# print the number of rows and columns
print(data.shape)
# New columns 
print(data.columns)

      zipCode countyCode  studyCount   STATE_NAME  PovertyEst  PovertyPercent  \
0       35616      28141         0.0  Mississippi        3335            16.0   
1       38827      28141         0.0  Mississippi        3335            16.0   
2       38326      28141         0.0  Mississippi        3335            16.0   
3       38846      28141         0.0  Mississippi        3335            16.0   
4       38873      28141         0.0  Mississippi        3335            16.0   
...       ...        ...         ...          ...         ...             ...   
21281   76930      48383         0.0        Texas       12193             8.5   
21282   76932      48383         0.0        Texas       12193             8.5   
21283   79755      48383         0.0        Texas       12193             8.5   
21284   79706      48383         0.0        Texas       12193             8.5   
21285   79739      48383         0.0        Texas       12193             8.5   

      medIncome         cou

In [145]:
# check for data types in the new dataframe and its structure
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21286 entries, 0 to 21285
Data columns (total 23 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   zipCode                                                       21286 non-null  object 
 1   countyCode                                                    21286 non-null  object 
 2   studyCount                                                    21286 non-null  float64
 3   STATE_NAME                                                    21286 non-null  object 
 4   PovertyEst                                                    21286 non-null  int64  
 5   PovertyPercent                                                21286 non-null  float64
 6   medIncome                                                     21286 non-null  object 
 7   countyName                                                    21286

In [146]:
# Rename columns with '_x' suffix to '_incd_x' and '_y' suffix to '_mort_y'
data = data.rename(columns={col: col.replace('_x', '_incd_x') for col in data.columns if col.endswith('_x')})
data = data.rename(columns={col: col.replace('_y', '_mort_y') for col in data.columns if col.endswith('_y')})

# For your specific example: 'recTrend_x' to 'recTrend_incd_x' and 'recTrend_y' to 'recTrend_mort_y'
data.rename(columns={'recTrend_x': 'recTrend_incd_x', 'recTrend_y': 'recTrend_mort_y'}, inplace=True)

In [147]:
# Renaming columns for easier access
column_renames = {
    'Age-Adjusted Incidence Rate([rate note]) - cases per 100,000': 'incidentRate',
    'Lower 95% Confidence Interval': 'Lower95CI',
    'Upper 95% Confidence Interval': 'Upper95CI',
    'CI*Rank([rank note])': 'CIRank',
    'Lower CI (CI*Rank)': 'LowerCIRank',
    'Upper CI (CI*Rank)': 'UpperCIRank',
    'Average Annual Count': 'AverageAnnualCount',
    'Recent 5-Year Trend ([trend note]) in Incidence Rates': 'Recent5YearTrend',
    'Lower 95% Confidence Interval.1': 'Lower95CI_1',
    'Upper 95% Confidence Interval.1': 'Upper95CI_1',
    'POPESTIMATE2022':'popEst2020', 
    'PovertyPercent':'povertyPercent'
}

data.rename(columns=column_renames, inplace=True)




In [148]:
# Display the first rows of the dataframe
data.head(5)

,zipCode,countyCode,studyCount,STATE_NAME,PovertyEst,povertyPercent,medIncome,countyName,popEst2020,incidentRate,...,LowerCIRank,UpperCIRank,AverageAnnualCount,recTrend_incd_x,Recent5YearTrend,Lower95CI_1,Upper95CI_1,deathRate,avgDeathsPerYear,recTrend_mort_y
0,35616,28141,0.0,Mississippi,3335,16.0,52635,Tishomingo County,18619,538.5,...,1,39,148,stable,3.5,0,8.1,192.3,54.0,falling
1,38827,28141,0.0,Mississippi,3335,16.0,52635,Tishomingo County,18619,538.5,...,1,39,148,stable,3.5,0,8.1,192.3,54.0,falling
2,38326,28141,0.0,Mississippi,3335,16.0,52635,Tishomingo County,18619,538.5,...,1,39,148,stable,3.5,0,8.1,192.3,54.0,falling
3,38846,28141,0.0,Mississippi,3335,16.0,52635,Tishomingo County,18619,538.5,...,1,39,148,stable,3.5,0,8.1,192.3,54.0,falling
4,38873,28141,0.0,Mississippi,3335,16.0,52635,Tishomingo County,18619,538.5,...,1,39,148,stable,3.5,0,8.1,192.3,54.0,falling


In [149]:
# Final merge data columns
data.columns

Index(['zipCode', 'countyCode', 'studyCount', 'STATE_NAME', 'PovertyEst',
       'povertyPercent', 'medIncome', 'countyName', 'popEst2020',
       'incidentRate', 'Lower95CI', 'Upper95CI', 'CIRank', 'LowerCIRank',
       'UpperCIRank', 'AverageAnnualCount', 'recTrend_incd_x',
       'Recent5YearTrend', 'Lower95CI_1', 'Upper95CI_1', 'deathRate',
       'avgDeathsPerYear', 'recTrend_mort_y'],
      dtype='object')

In [150]:
# New data structure
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21286 entries, 0 to 21285
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   zipCode             21286 non-null  object 
 1   countyCode          21286 non-null  object 
 2   studyCount          21286 non-null  float64
 3   STATE_NAME          21286 non-null  object 
 4   PovertyEst          21286 non-null  int64  
 5   povertyPercent      21286 non-null  float64
 6   medIncome           21286 non-null  object 
 7   countyName          21286 non-null  object 
 8   popEst2020          21286 non-null  int64  
 9   incidentRate        21286 non-null  object 
 10  Lower95CI           21286 non-null  object 
 11  Upper95CI           21286 non-null  object 
 12  CIRank              21286 non-null  object 
 13  LowerCIRank         21286 non-null  object 
 14  UpperCIRank         21286 non-null  object 
 15  AverageAnnualCount  21286 non-null  object 
 16  recT

In [151]:
# Extract numeric values from 'incidentRate'
for i, j in enumerate(data['incidentRate']):
    try:
        # Find the first occurrence of a numeric pattern
        data.loc[i, 'incidentRate'] = re.findall('\d+\.\d+|\d+', j)[0]
    except:
        # Set to NaN if no numeric pattern is found
        data.loc[i, 'incidentRate'] = np.nan

# Convert 'incidentRate' to float
data['incidentRate'] = data['incidentRate'].astype('float64')

# Convert 'medIncome' to float
data['medIncome'] = data['medIncome'].astype('float64')

# Convert 'AverageAnnualCount' to float, handling non-numeric cases
for i, j in enumerate(data['AverageAnnualCount']):
    try:
        data.loc[i, 'AverageAnnualCount'] = float(j)
    except:
        data.loc[i, 'AverageAnnualCount'] = np.nan

            

In [152]:
# Convert 'popEst2020' to float64 type
data['popEst2020'] = data['popEst2020'].astype('float64')

# Convert 'povertyPercent' to float64 type
data['povertyPercent'] = data['povertyPercent'].astype('float64')

# Calculate and print the number of NaN values in specific columns

# Count and print the number of NaN values in 'incidentRate'
incidence_rate_nan_count = data['incidentRate'].isnull().sum()
print('incidentRate # nan:', incidence_rate_nan_count)

# Count and print the number of NaN values in 'medIncome'
med_income_nan_count = data['medIncome'].isnull().sum()
print('medIncome # nan:', med_income_nan_count)

# Count and print the number of NaN values in 'AverageAnnualCount'
avg_ann_count_nan_count = data['AverageAnnualCount'].isnull().sum()
print('AverageAnnualCount # nan:', avg_ann_count_nan_count)

# Count and print the number of NaN values in 'popEst2020'
pop_est_2020_nan_count = data['popEst2020'].isnull().sum()
print('popEst2020 # nan:', pop_est_2020_nan_count)


incidentRate # nan: 3631
medIncome # nan: 0
AverageAnnualCount # nan: 3631
popEst2020 # nan: 0


In [153]:
# Fill in the missing values with mean
data['incidentRate'].fillna(value=data['incidentRate'].mean(), inplace=True)
data['AverageAnnualCount'].fillna(value=data['AverageAnnualCount'].mean(), inplace=True)

In [154]:

# Count the number of duplicate zip codes and counties
num_duplicate_zips = data['zipCode'].duplicated().sum()
num_duplicate_counties = data['countyCode'].duplicated().sum()

# Print the shape of the DataFrame and the count of duplicates and unique values
print("data:")
print(data.shape)
print(f"# of duplicate zips: {num_duplicate_zips}")
print(f"# of duplicate counties: {num_duplicate_counties}")
print(f"# unique zips: {data['zipCode'].nunique()}")
print(f"# unique counties: {data['countyCode'].nunique()}")
print(data.columns)


data:
(21286, 23)
# of duplicate zips: 7119
# of duplicate counties: 20184
# unique zips: 14167
# unique counties: 1102
Index(['zipCode', 'countyCode', 'studyCount', 'STATE_NAME', 'PovertyEst',
       'povertyPercent', 'medIncome', 'countyName', 'popEst2020',
       'incidentRate', 'Lower95CI', 'Upper95CI', 'CIRank', 'LowerCIRank',
       'UpperCIRank', 'AverageAnnualCount', 'recTrend_incd_x',
       'Recent5YearTrend', 'Lower95CI_1', 'Upper95CI_1', 'deathRate',
       'avgDeathsPerYear', 'recTrend_mort_y'],
      dtype='object')


In [155]:
# Final data structure
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21286 entries, 0 to 21285
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   zipCode             21286 non-null  object 
 1   countyCode          21286 non-null  object 
 2   studyCount          21286 non-null  float64
 3   STATE_NAME          21286 non-null  object 
 4   PovertyEst          21286 non-null  int64  
 5   povertyPercent      21286 non-null  float64
 6   medIncome           21286 non-null  float64
 7   countyName          21286 non-null  object 
 8   popEst2020          21286 non-null  float64
 9   incidentRate        21286 non-null  float64
 10  Lower95CI           21286 non-null  object 
 11  Upper95CI           21286 non-null  object 
 12  CIRank              21286 non-null  object 
 13  LowerCIRank         21286 non-null  object 
 14  UpperCIRank         21286 non-null  object 
 15  AverageAnnualCount  21286 non-null  float64
 16  recT

In [156]:
# Full Data csv file
full_data = data[['zipCode','countyCode','countyName','STATE_NAME','PovertyEst','povertyPercent',
     'medIncome','popEst2020','incidentRate','AverageAnnualCount','recTrend_incd_x','recTrend_mort_y','Recent5YearTrend']]

In [157]:
# Save a csv file
full_data.to_csv('fulldata.csv', index=False)

In [158]:
# Group the data by countyCode
countyData = pd.pivot_table(data, index=['countyCode'])

# Reset the index to convert the grouped column 'countyCode' back to a regular column
countyData.reset_index(inplace=True)

# Create a separate pivot table for summing the 'studyCount' column for each county
t = pd.pivot_table(data, index=['countyCode'], aggfunc='sum')

# Drop the 'studyCount' column from the 'countyData' DataFrame
countyData.drop('studyCount', axis=1, inplace=True)

# Assign the 'studyCount' values from the 't' DataFrame to the 'countyData' DataFrame
countyData = countyData.assign(studyCount=[i for i in t.studyCount])

# Calculate the 'studyPerCap' column by dividing 'studyCount' by 'popEst2020' and scaling by 1 million
countyData['studyPerCap'] = (countyData['studyCount'] / countyData['popEst2020']) * 1e6


C:\Users\chimi\AppData\Local\Temp\ipykernel_18828\1648361814.py:2: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  countyData = pd.pivot_table(data, index=['countyCode'])
C:\Users\chimi\AppData\Local\Temp\ipykernel_18828\1648361814.py:8: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  t = pd.pivot_table(data, index=['countyCode'], aggfunc='sum')


In [159]:
# Preview of the county_data
countyData.head(5)

,countyCode,AverageAnnualCount,PovertyEst,avgDeathsPerYear,deathRate,incidentRate,medIncome,popEst2020,povertyPercent,studyCount,studyPerCap
0,12101,4036.0,165851,1401.0,161.6,499.9,76713.0,608794.0,11.1,36.0,59.133303
1,12103,7624.0,65148,2519.0,143.4,476.9,65933.0,961739.0,10.9,168.0,174.683568
2,12105,4591.0,117299,1485.0,146.0,473.1,66427.0,787404.0,12.4,37.0,46.989855
3,12107,559.0,120353,232.0,192.4,487.2,61941.0,74731.0,15.7,4.0,53.525311
4,12109,1768.0,16484,493.0,132.8,488.8,46459.0,306841.0,22.5,165.0,537.737786


In [161]:
# Studies per 1,000,000 people
zipCodeData = pd.pivot_table(data, index=['zipCode'])
zipCodeData.reset_index(inplace=True)
zipCodeData['studyPerCap'] = (zipCodeData['studyCount'] / zipCodeData['popEst2020'] * 10**6)


C:\Users\chimi\AppData\Local\Temp\ipykernel_18828\3968452472.py:2: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  zipCodeData = pd.pivot_table(data, index=['zipCode'])


In [162]:
zipCodeData.head(5)
# Save it as csv file
zipCodeData.to_csv('zipCodedata.csv',index=False)

In [163]:
# Studies per 1,000,000 people
duplicated_zip_codes = zipCodeData['zipCode'].duplicated().sum()
total_unique_zip_codes = len(zipCodeData['zipCode'].unique())

print("zipCodeData:")
print(f"Shape: {zipCodeData.shape}")
print(f"# of duplicate zips: {duplicated_zip_codes}")
print(f"# unique zips: {total_unique_zip_codes}")
print("Columns:", zipCodeData.columns)


zipCodeData:
Shape: (14167, 11)
# of duplicate zips: 0
# unique zips: 14167
Columns: Index(['zipCode', 'AverageAnnualCount', 'PovertyEst', 'avgDeathsPerYear',
       'deathRate', 'incidentRate', 'medIncome', 'popEst2020',
       'povertyPercent', 'studyCount', 'studyPerCap'],
      dtype='object')


In [164]:
# Looking at duplicated counties 
print("countyData:")
print(f"Shape: {countyData.shape}")
duplicated_counties = countyData['countyCode'].duplicated().sum()
total_unique_counties = len(countyData['countyCode'].unique())
print(f"# of duplicate counties: {duplicated_counties}")
print(f"# unique counties: {total_unique_counties}")
print("Columns:", countyData.columns)


countyData:
Shape: (1102, 11)
# of duplicate counties: 0
# unique counties: 1102
Columns: Index(['countyCode', 'AverageAnnualCount', 'PovertyEst', 'avgDeathsPerYear',
       'deathRate', 'incidentRate', 'medIncome', 'popEst2020',
       'povertyPercent', 'studyCount', 'studyPerCap'],
      dtype='object')


In [171]:

# Define the quantiles for discretization
quantiles = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Create a DataFrame 'countyData' containing your data

# Calculate the income bins for 'countyData' based on the specified quantiles
countyCuts = countyData['medIncome'].quantile(quantiles).values

# Discretize 'medIncome' using the calculated bins
countyBinnedInc = pd.cut(countyData['medIncome'], bins=countyCuts, right=True, include_lowest=True)


# Count the occurrences of each bin
bin_value_counts = countyBinnedInc.value_counts()

# Assign the discretized income column back to 'countyData'
countyData['binnedInc'] = countyBinnedInc

# Group 'countyData' by the 'binnedInc' column and calculate the mean for each group
countyBinned = countyData.groupby(['binnedInc']).mean()

# Reset the index to make 'binnedInc' a regular column
countyBinned = countyBinned.reset_index()


C:\Users\chimi\AppData\Local\Temp\ipykernel_18828\456505981.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  countyBinned = countyData.groupby(['binnedInc']).mean()


In [172]:

# Define the quantiles for discretization
quantiles = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Create a DataFrame 'zipCodeData' containing your data

# Calculate the income bins for 'zipCodeData' based on the specified quantiles
zipCuts = zipCodeData['medIncome'].quantile(quantiles).values

# Discretize 'medIncome' using the calculated bins
zipCodeData['binnedInc'] = pd.cut(zipCodeData['medIncome'], bins=zipCuts, right=True, include_lowest=True)

# Count the occurrences of each bin
bin_value_counts = zipCodeData['binnedInc'].value_counts()

# Group 'zipCodeData' by the 'binnedInc' column and calculate the mean for each group
zipBinned = zipCodeData.groupby('binnedInc').mean().reset_index()


C:\Users\chimi\AppData\Local\Temp\ipykernel_18828\702283007.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  zipBinned = zipCodeData.groupby('binnedInc').mean().reset_index()


In [173]:

# Extract relevant columns from 'countyData'
death_rate = countyData['deathRate']
incidence_rate = countyData['incidentRate']
binned_income = countyData['binnedInc']
population_estimate = countyData['popEst2020']

# Create lists for the data
cancer = list(death_rate) + list(incidence_rate)
cancer_labels = ['Death Rate'] * len(death_rate) + ['Incidence Rate'] * len(incidence_rate)
bInc = list(binned_income) * 2
popEst = list(population_estimate) * 2

# Create the DataFrame 'iddf'
iddf = pd.DataFrame({'cancer': cancer, 'cancerLabels': cancer_labels, 'bInc': bInc, 'popEst': popEst})


In [174]:
# Convert countyData and and ZipCodeData to csv files
countyData.to_csv('county_data.csv',index=False)
zipCodeData.to_csv('zipCodedata.csv',index=False)